## 문제. 달리기 경주

[바로가기](https://school.programmers.co.kr/learn/courses/30/lessons/178871)


## 풀이 방법

### 1) 초기 방법

- 제한 사항에 `5 ≤ players의 길이 ≤ 50,000`으로 보아 배열에 길의가 길어지 경우 단순히 for문을 통해서 계속 찾는 방식은 시간 복잡도에 영향을 줄 것으로 판단하였다.
- 따라서, 처음에는 `이진 트리`의 방식을 착안하여 리스트를 반으로 나누게 된다면, 최대 $O$$(n)$의 n이 절반으로 줄게 된다면, 시간을 단축시킬 수 있을 것이라 판단
- 하지만, 결과적으로 단점이 있다면 결국 `index`로 탐색하는 방식이 었고, 이는 사이즈가 작은 배열을 탐색하기에는 유리한 방식이지만, 사이즈가 클 수록 시간이 많이 든다는 단점이 있었다.
- 결국 7~13의 테스트 케이스에서 `시간 초과`가 발생

### 초기 방법 풀이

In [ ]:
def solution(players, callings):
    answer = []
    left, right = players[:len(players)//2] , players[len(players)//2:]
    
    for c in callings: 
        if c in left: 
            targetIndex = left.index(c)
            if targetIndex:
                left[targetIndex-1],left[targetIndex]=left[targetIndex],left[targetIndex-1]
                
        if c in right: 
            targetIndex = right.index(c)
            if targetIndex >= 1:
                right[targetIndex-1],right[targetIndex]=right[targetIndex],right[targetIndex-1]
            else:
                go_left = right.pop(0)
                go_right = left.pop(-1)
                left.append(go_left)
                right.insert(0,go_right)
    return left+right

### What I learn?

#### 인덱스는 무조건 좋을까?

- 인덱스를 사용하면, 전반적인 시스템 전체의 부하를 줄일 수 있다. 하지만, 이런 좋은 인덱스도 제대로 사용하지 못하면 역효과를 볼 수 있다.
- 또한 앞서 B-Tree를 이야기하며 말한것 처럼 조회 성능은 증가할 수 있으나, 생성/수정/삭제 작업에 대한 성능은 오히려 낮아진다. 오히려 전체 성능이 저하될 수 있다.

### 2) 해시 적용 풀이

## 풀이 방법

### 2) 해시 테이블 적용 풀이

- 배열을 탐색하는 것에서 시간이 오래 걸린다면, `해시`로 바꿔서 인덱스와 값을 바꾼 구조를 사용하는 것이 더 나을 것이라 판단하여 해시를 적용.
- 하지만, 똑같이 `7~13` 테스트 케이스에서 `시간 초과`가 발생
- 가장 마음에 걸렸던 부분이 `등수`를 찾기 위해 for문을 사용해야 한다는 것이 $O$$(n)$ * callings의 수 만큼 발생할 것 같다는 느낌이 들었다.
- 그렇다면 관건은 `등수`를 찾는 것에서 `for`를 사용하지 않고 시간 복잡도를 줄일 수 있는 방법을 찾아야 했다.

In [ ]:
def solution(players, callings):
    hash_players = {player : idx+1 for idx,player in enumerate(players)}
    hash_players_idx = {idx+1:player for idx,player in enumerate(players)}

    for c in callings: 
        front_player_score = hash_players[c] - 1
        rear_player_score = hash_players[c] + 1
        
        front_player = [key for key, value in hash_players.items() if value == front_player_score][0]
        
        hash_players[c] -=1
        hash_players[front_player]+=1
    
    return  sorted(hash_players,key=lambda x:hash_players[x] )

### What I learn?

#### 해시 테이블을 적절하게 사용하기 위해서

- 단순히 해시 테이블을 사용해서 pop과 insert/append를 통해서 CRUD를 하는 것이 아닌,
- 등수와 선수를 역순으로 바꾼 테이블과 반대로 기존의 선수의 위치를 바로 알 수 있도록 선수와 등수를 바꾼 테이블 
- 즉, 두 개의 테이블이 필요하고, 이를 선택한 선수의 등수를 -1 하고 조회를 위해서 다른 해시 테이블에서 앞 선수의 등수를 조회하면 풀이가 가능해진다.

In [ ]:
# 포인트는 insert()와 pop() 또는 remove()등을 쓰지 않는 것이다. 
def solution(players, callings):
    # 선수: 위치
    p_idx_dict = {player: i for i, player in enumerate(players)}
    # 위치: 선수
    idx_p_dict = {i: player for i, player in enumerate(players)}
    
    for call in callings:
        cur_idx = p_idx_dict[call]  # 현재 선수의 위치
        pre_idx = cur_idx-1         # 현재 선수보다 앞에 있는 선수 등수
        cur_player = call
        pre_player = idx_p_dict[pre_idx]    # *

        p_idx_dict[cur_player] = pre_idx    # 현재 선수는 앞에 선수 등수가 되고
        p_idx_dict[pre_player] = cur_idx    # 앞에 선수는 뒷 등수가 되고 
        
        idx_p_dict[pre_idx] = cur_player    # 등수별 선수 표 업데이트 * 때문에
        idx_p_dict[cur_idx] = pre_player

    return list(idx_p_dict.values())